In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config['project_path'] = '..'
config['data_folder_path'] = 'data'
config['temp_folder_path'] = 'temp'

# input files
config['BHT input'] = 'CRM_data_0713.xlsx'
config['MDM_mapping'] = 'MDM_mapping.xlsx'

# output files
config['spending_CRM'] = 'spending CRM 202001-04.xlsx'
config['performance_CRM'] = 'performance CRM 202001-04.xlsx'
config['performance_search_index'] = 'performance search index 202001-04.xlsx'

## Read CRM

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['BHT input'] 

spending_input_df = pd.read_excel(input_file_path, sheet_name='spending', dtype=str)

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['BHT input'] 

perform_input_df = pd.read_excel(input_file_path, sheet_name='KPI', dtype=str)
perform_input_df['Value'] = perform_input_df['KPI_value'].astype('float')

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['MDM_mapping']

mdm_input_df = pd.read_excel(input_file_path, dtype=str)
mdm_input_df = mdm_input_df[['Province_code', 'MDM_province_code']].drop_duplicates()

In [ ]:
def regulate_shannxi_shaanxi(province_code):
    if province_code == 'shaanxi':
        return 'shannxi'
    return province_code

In [ ]:
spending_input_df['Province'] = spending_input_df['Province'].apply(lambda x: regulate_shannxi_shaanxi(x))
spending_df = pd.merge(spending_input_df, mdm_input_df, left_on='Province', right_on='Province_code', how='left')

In [ ]:
perform_input_df['Province'] = perform_input_df['Province'].apply(lambda x: regulate_shannxi_shaanxi(x))
perform_df = pd.merge(perform_input_df, mdm_input_df, left_on='Province', right_on='Province_code', how='left')

### Check for um-mapped records

In [ ]:
spending_df[spending_df['MDM_province_code'].isnull()]

In [ ]:
perform_df[perform_df['MDM_province_code'].isnull()]

In [ ]:
spending_df

In [ ]:
output_df = spending_df[['Date', 'MDM_province_code', 'Cost_type_1', 'Cost_type_2', 'Cost_type_3', 'Cost_type_4', 'AIP', 'Brand', 'Spending_value']]
output_df.columns = ['Date', 'Province', 'Cost_type_1', 'Cost_type_2', 'Cost_type_3', 'Cost_type_4', 'AIP', 'Brand', 'Spending_value']

## Output CRM Investment

In [ ]:
output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['spending_CRM']

output_df.to_excel(output_file_path, index=False)

## Output CRM Performance

In [ ]:
perform_df = perform_df[['Date','MDM_province_code', 'Brand', 'KPI', 'KPI_value']]
perform_df.columns = ['Date','Province', 'Brand', 'KPI', 'KPI_value']

In [ ]:
perf_crm_df = perform_df[(perform_df['KPI'] == 'CRM Leads Offline') 
                       | (perform_df['KPI'] == 'CRM Leads Online')
                       | (perform_df['KPI'] == 'CRM NU Offline')
                       | (perform_df['KPI'] == 'CRM NU Online')
                       | (perform_df['KPI'] == 'Active Leads')
                       | (perform_df['KPI'] == 'Landing UV')].copy()

output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['performance_CRM']

perf_crm_df.to_excel(output_file_path, index=False)

In [ ]:
perf_si_df = perform_df[(perform_df['KPI'] == 'Ali Search')
                       | (perform_df['KPI'] == 'Baidu Search')].copy()

output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['performance_search_index']

perf_si_df.to_excel(output_file_path, index=False)